In [3]:
import pandas as pd


In [112]:
exp_list= [
      {
        "id": "fe352289a25a97ea9b53d1945d1558b9",
        "user_id": "string",
        "amount": 1000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-11-07"
      },
      {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 3000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-11-05"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 3000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-11-03"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 1500,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-11-05"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 300,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2019-12-30"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 4000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-10-05"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 14000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-09-05"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 4100,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-07-05"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 4000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-06-05"
      },
    {
        "id": "05753e3e926320eb2364c9da8df77c35",
        "user_id": "string",
        "amount": 4000,
        "description": "string",
        "comments": "string",
        "payor": "string",
        "date": "2020-04-05"
      }
]

In [21]:
df = pd.DataFrame(exp_list)
df

,id,user_id,amount,description,comments,payor,date
0,fe352289a25a97ea9b53d1945d1558b9,string,1000,string,string,string,2020-11-07
1,05753e3e926320eb2364c9da8df77c35,string,3000,string,string,string,2020-11-05
2,05753e3e926320eb2364c9da8df77c35,string,3000,string,string,string,2020-11-03
3,05753e3e926320eb2364c9da8df77c35,string,1500,string,string,string,2020-11-05
4,05753e3e926320eb2364c9da8df77c35,string,300,string,string,string,2019-12-30
5,05753e3e926320eb2364c9da8df77c35,string,4000,string,string,string,2020-10-05
6,05753e3e926320eb2364c9da8df77c35,string,14000,string,string,string,2020-09-05
7,05753e3e926320eb2364c9da8df77c35,string,4100,string,string,string,2020-07-05
8,05753e3e926320eb2364c9da8df77c35,string,4000,string,string,string,2020-06-05
9,05753e3e926320eb2364c9da8df77c35,string,4000,string,string,string,2020-04-05


In [22]:
from datetime import datetime, timedelta,date
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.year.astype('str') + '-' + df['date'].dt.month.astype('str') + '-01'
df['date'] = pd.to_datetime(df['date'])#groupby date and sum the sales
df = df.groupby('date').amount.sum().reset_index()
df

,date,amount
0,2019-12-01,300
1,2020-04-01,4000
2,2020-06-01,4000
3,2020-07-01,4100
4,2020-09-01,14000
5,2020-10-01,4000
6,2020-11-01,8500


In [14]:
%matplotlib inline
import matplotlib.pyplot as plt

#import Keras
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers import LSTM
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [23]:
#create a new dataframe to model the difference
df_diff = df.copy()#add previous sales to the next row
df_diff['prev_amount'] = df_diff['amount'].shift(1)#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['amount'] - df_diff['prev_amount'])
df_diff

,date,amount,prev_amount,diff
1,2020-04-01,4000,300.0,3700.0
2,2020-06-01,4000,4000.0,0.0
3,2020-07-01,4100,4000.0,100.0
4,2020-09-01,14000,4100.0,9900.0
5,2020-10-01,4000,14000.0,-10000.0
6,2020-11-01,8500,4000.0,4500.0


In [24]:

#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_amount'],axis=1)#adding lags
for inc in range(1,4):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)
df_supervised

,date,amount,diff,lag_1,lag_2
0,2020-07-01,4100,100.0,0.0,3700.0
1,2020-09-01,14000,9900.0,100.0,0.0
2,2020-10-01,4000,-10000.0,9900.0,100.0
3,2020-11-01,8500,4500.0,-10000.0,9900.0


In [25]:
# Import statsmodels.formula.api
import statsmodels.formula.api as smf# Define the regression formula
model = smf.ols(formula='diff ~ lag_1', data=df_supervised)# Fit the regression
model_fit = model.fit()# Extract the adjusted r-squared
regression_adj_rsq = model_fit.rsquared_adj
print(regression_adj_rsq)

0.22662868489529997


In [26]:
#import MinMaxScaler and create a new dataframe for LSTM model
from sklearn.preprocessing import MinMaxScaler
df_model = df_supervised.drop(['amount','date'],axis=1)#split train and test set
train_set, test_set = df_model[0:-2].values, df_model[-2:].values

In [27]:
#apply Min Max Scaler
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [30]:
X_train= train_set_scaled[:, 1:]
y_train =  train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

In [32]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1, shuffle=False)

Epoch 1/10
2/2 [==============================] - 0s 1ms/step - loss: 0.9926
Epoch 2/10
2/2 [==============================] - 0s 2ms/step - loss: 0.9754
Epoch 3/10
2/2 [==============================] - 0s 1ms/step - loss: 0.9635
Epoch 4/10
2/2 [==============================] - 0s 1ms/step - loss: 0.9552
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 0.9486
Epoch 6/10
2/2 [==============================] - 0s 1ms/step - loss: 0.9428
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.9373
Epoch 8/10
2/2 [==============================] - 0s 1ms/step 

In [33]:
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

y_pred = model.predict(X_test,batch_size=1)#for multistep prediction, you need to replace X_test values with the predictions coming from t-1

In [34]:
import numpy as np
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

[[ -0.65884137 197.          -0.94594595]]
[[   0.73503917 -201.            4.35135135]]


In [35]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df[-7:].date)
act_sales = list(df[-7:].amount)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)#for multistep prediction, replace act_sales with the predicted sales

In [37]:
df_result

,pred_value,date
0,2071,2020-04-01
1,12601,2020-06-01


In [51]:
from core.transformer import XmlTransformer
data = {
   "point": [
      {
         "x": "Jan",
         "y": "100"
      },
      {
         "x": "Feb",
         "y": "200"
      },
      {
         "x": "Mar",
         "y": "300"
      },
      {
         "x": "Apr",
         "y": "400"
      },
      {
         "x": "May",
         "y": "500"
      }
   ],
}
d = {"data":data, "EndIndex": "3"}
XmlTransformer(d)

<?xml version="1.0" ?>
<response>
<data>
<point>
<x>Jan</x>
<y>100</y>
</point>
<point>
<x>Feb</x>
<y>200</y>
</point>
<point>
<x>Mar</x>
<y>300</y>
</point>
<point>
<x>Apr</x>
<y>400</y>
</point>
<point>
<x>May</x>
<y>500</y>
</point>
</data>
<EndIndex>3</EndIndex>
</response>

In [83]:
df_result_1 = df_result.copy()
df_result_1["x"]= df_result['date']
df_result_1["y"]= df_result['pred_value']
df_result_1 = df_result_1.drop(columns=["date","pred_value"],axis=0)
df_result_1.to_dict(orient='records')

[{'x': Timestamp('2020-04-01 00:00:00'), 'y': 2071},
 {'x': Timestamp('2020-06-01 00:00:00'), 'y': 12601}]

# For API

In [113]:
df = pd.DataFrame(exp_list)
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.year.astype('str') + '-' + df['date'].dt.month.astype('str') + '-01'
df['date'] = pd.to_datetime(df['date'])#groupby date and sum the sales
df = df.groupby('date').amount.sum().reset_index()

In [114]:
latest_dt = df['date'].max()

In [115]:
datetime_series = pd.Series(
    pd.date_range(latest_dt, periods=6, freq="M")
)
df_test = pd.DataFrame()
df_test['date'] = datetime_series
df_test['amount'] = 0
df_test

,date,amount
0,2020-11-30,0
1,2020-12-31,0
2,2021-01-31,0
3,2021-02-28,0
4,2021-03-31,0
5,2021-04-30,0


In [118]:
#create a new dataframe to model the difference
df_diff = df.copy()
df_diff['prev_amount'] = df_diff['amount'].shift(1)#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['amount'] - df_diff['prev_amount'])
df_supervised = df_diff.drop(['prev_amount'],axis=1)#adding lags
#add previous sales to the next row
for inc in range(1,4):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)#drop null values

df_supervised = df_supervised.dropna().reset_index(drop=True)
df_supervised

,date,amount,diff,lag_1,lag_2,lag_3
0,2020-09-01,14000,9900.0,100.0,0.0,3700.0
1,2020-10-01,4000,-10000.0,9900.0,100.0,0.0
2,2020-11-01,8500,4500.0,-10000.0,9900.0,100.0


In [121]:
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

df_model = df_supervised.drop(['amount','date'],axis=1)#split train and test set
# train_set, test_set = df_model[0:-3].values, df_model[-3:].values
train_set = df_model.values
test_set = df_test.values

scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train_set)
# reshape training set
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled = scaler.transform(train_set)
# reshape test set
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)


X_train= train_set_scaled[:, 1:]
y_train =  train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [ ]:
model = Sequential()
model.add(LSTM(4, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1, shuffle=False)

In [ ]:
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

y_pred = model.predict(X_test,batch_size=1)#for multistep prediction, you need to replace X_test values with the predictions coming from t-1

In [ ]:
import numpy as np
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])#rebuild test set for inverse transform
pred_test_set = []
for index in range(0,len(y_pred)):
    print(np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))#reshape pred_test_set
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])#inverse transform
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)

In [106]:
df_model


,diff,lag_1,lag_2
0,100.0,0.0,3700.0
1,9900.0,100.0,0.0
2,-10000.0,9900.0,100.0
3,4500.0,-10000.0,9900.0
